### Add scripts to the notebooks path

In [12]:
import os, sys

current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

scripts_path = os.path.join(parent_dir, 'scripts')

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

# Insert the path to the Scripts directory
sys.path.insert(0, scripts_path)

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

d:\KifiyaAIM-Course\Week - 10\Brent-Oil-Market-Dynamics\notebooks


### Import Statements

In [13]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:
sns.set_theme()

### Data Collection

The data has already been provided and is present offline so the loading will be considered as the collection step

In [15]:
DATA_PATH = "../data/Brent_Oil_Prices.csv"
data = pd.read_csv(DATA_PATH)

### Exploratory Data Analysis

In [16]:
from scripts.exploratory_data_analyzer import EDAAnalyzer

analyzer = EDAAnalyzer(data=data)

1) Basic overview of the data

In [17]:
analyzer.basic_overview()

The data has a shape of: (9011, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9011 entries, 0 to 9010
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    9011 non-null   object 
 1   Price   9011 non-null   float64
dtypes: float64(1), object(1)
memory usage: 140.9+ KB
None


From the result it can be seen that the date column is being treated as a string and not a date object so I will convert it to a date object

In [18]:
data['Date'] = pd.to_datetime(data['Date'])

2) Summary Statistics

In [19]:
analyzer.summary_statistics()

                                Date        Price
count                           9011  9011.000000
mean   2005-02-10 23:46:53.760958720    48.420782
min              1987-05-20 00:00:00     9.100000
25%              1996-03-20 12:00:00    19.050000
50%              2005-02-02 00:00:00    38.570000
75%              2014-01-09 12:00:00    70.090000
max              2022-11-14 00:00:00   143.950000
std                              NaN    32.860110


***Observations:***

- The data starts of on **1987-05-20** and continues upto **2022-11-14** which comes to 35+ years of data.
- The oil price shows a standard deviation of 32.86$ which is big but considering it is over a period of 35+ years it isn't that exagurated